In [1]:
with open('06.txt') as f:
    grid = [line.strip() for line in f.readlines()]

len(grid), len(grid[0])

(130, 130)

# Part 1: Positions

In [2]:
DIRECTIONS = [
    # line, col
    (-1,  0), # up
    ( 0,  1), # right
    (+1,  0), # down
    ( 0, -1), # left
]
from itertools import cycle

def find_char(array: list[str], search: str):
    for y, line in enumerate(array):
        for x, char in enumerate(line):
            if char == search:
                return x, y

Because we want to count the **unique** positions - including crossovers! - our options are to either
- replace each character with an `'X'` and count at the end
- record a `set` of all coordinates crossed

The two are probably pretty comparable on grounds of efficiency. The former is probably easier to debug, but the latter is slightly easier to code, so we'll do that.

We're going to make the **assumption** that the guard eventually leaves the boundary, so we won't check for if otherwise.

In [3]:
def walk(array):
    x, y = find_char(array, '^')
    coords_seen = {(x, y)}
    for dy, dx in cycle(DIRECTIONS):
        # print(dx, dy)
        while array[y+dy][x+dx] != '#':
            y += dy
            x += dx
            coords_seen.add((x, y))
            if not (0 <= y+dy < len(array)):
                # print('y', x, y)
                return coords_seen
            if not (0 <= x+dx < len(array[y])):
                # print('x', x, y)
                return coords_seen

path = walk(grid)
print(len(path))

# grid2 = [list(line) for line in grid]
# for x, y in path:
#     grid2[y][x] = grid2[y][x].replace('.', 'X')

# print('\n'.join(''.join(line) for line in grid2))

4964


# Part 2: Positions we could cause a loop from

It's self-evident that we can only cause loops by putting a block in one of the positions on `path`. That's great! It means we need only try them all and see if there is a loop.

How we test for a loop isn't too difficult: we'll store a set of `(x, y, direction)` tuples and, if we see a duplicate, we've found ourselves a loop. Note that, as the examples show, the loop might not include the starting position!

In [4]:
def does_loop(array: list[str]):
    start_pos = find_char(array, '^')
    if start_pos is None:
        # oops! we replaced the guard with an obstacle
        return False
    x, y = start_pos

    coords_seen = set()
    for dy, dx in cycle(DIRECTIONS):
        while array[y+dy][x+dx] != '#':
            position = (x, y, dy, dx)
            if position in coords_seen:
                return True
            coords_seen.add(position)
            y += dy
            x += dx
            if not (0 <= y+dy < len(array)):
                return False
            if not (0 <= x+dx < len(array[y])):
                return False

does_loop(grid)

def add_obstacle(array, x, y):
    array2 = list(array)
    line = list(array2[y])
    line[x] = '#'
    array2[y] = ''.join(line)
    return array2

sum(does_loop(add_obstacle(grid, x, y)) for (x, y) in path)

1740